In [1]:
from mxnet import nd
import random
import zipfile
import collections
import d2lzh as d2l
import math
from mxnet import autograd,gluon,nd
from mxnet.gluon import data as gdata,loss as gloss,nn,utils as gutils
import os
import tarfile
import sys
import time
import numpy as np
from mxnet import init
from mxnet.contrib import text

In [2]:
def read_embedding_params(filePath):
    idx_to_token=np.load(filePath+'idx_to_token.npy')
    #token_to_idx=np.load(filePath+'token_to_idx.npy')
    token_to_idx={tk:idx for idx,tk in enumerate(idx_to_token)}
    embedding_weight=nd.load(filePath+'embedding_weight.txt')
    return idx_to_token,token_to_idx,embedding_weight
def save_params(params,filePath):
    file=filePath
    nd.save(file+'conv_layer_one_W.txt',params[0])
    nd.save(file+'conv_layer_one_B.txt',params[1])
    nd.save(file+'conv_layer_two_W.txt',params[2])
    nd.save(file+'conv_layer_two_B.txt',params[3])
    nd.save(file+'conv_layer_three_W.txt',params[4])
    nd.save(file+'conv_layer_three_B.txt',params[5])
    nd.save(file+'conv_layer_four_W.txt',params[6])
    nd.save(file+'conv_layer_four_B.txt',params[7])
    nd.save(file+'ful_layer_connect_W.txt',params[8])
    nd.save(file+'ful_layer_connect_B.txt',params[9])
    return
def init_params():
    conv_one_W=nd.zeros(shape=(20,3))
    conv_one_W=nd.random.normal(scale=0.01,shape=(conv_one_W.shape))
    conv_one_B=nd.zeros(shape=(3,3))
    conv_one_B=nd.random.normal(scale=0.01,shape=(conv_one_B.shape))
    
    conv_two_W=nd.zeros(shape=(9,3))
    conv_two_W=nd.random.normal(scale=0.01,shape=(conv_two_W.shape))
    conv_two_B=nd.zeros(shape=(3,3))
    conv_two_B=nd.random.normal(scale=0.01,shape=(conv_two_B.shape))

    conv_three_W=nd.zeros(shape=(9,3))
    conv_three_W=nd.random.normal(scale=0.01,shape=(conv_three_W.shape))
    conv_three_B=nd.zeros(shape=(3,3))
    conv_three_B=nd.random.normal(scale=0.01,shape=(conv_three_B.shape))

    conv_four_W=nd.zeros(shape=(9,1))
    conv_four_W=nd.random.normal(scale=0.01,shape=(conv_four_W.shape))
    conv_four_B=nd.zeros(1)
    conv_four_B=nd.random.normal(scale=0.01,shape=(conv_four_B.shape))

    full_connect_W=nd.zeros(shape=(100,80))
    full_connect_W=nd.random.normal(scale=0.01,shape=(full_connect_W.shape))
    full_connect_B=nd.zeros(shape=(80,1))
    full_connect_B=nd.random.normal(scale=0.01,shape=(full_connect_B.shape))
        #print(conv_one_W)
    params=[conv_one_W,conv_one_B,conv_two_W,conv_two_B,conv_three_W,conv_three_B,conv_four_W,conv_four_B,full_connect_W,full_connect_B]
    return params
def read_file_data(filePath,token_to_idx,text_size):
    f=open(filePath)
    lines=f.readlines()
    raw_dataset=[st.split() for st in lines]
    raw_dataset.remove([])
    wordNum=0;
    dataset_temp=[]
    for st in raw_dataset:
         for wt in st:
                wordNum+=1
    dataset=[[token_to_idx[tk] for tk in st if tk in token_to_idx] for st in raw_dataset]
    dict_size=text_size #文本大小
    dataset=np.array(dataset)
    temp_one=np.array([])
    for row in dataset:
        for col in row:
            temp_one=np.append(temp_one,np.array(col))
    zeros=np.zeros(dict_size-wordNum)
    temp_two=np.append(temp_one,zeros)
    temp_three=np.array(temp_two)
    #print(temp_three)
    temp_three=temp_three.reshape(dict_size,-1)
    return temp_three
def read_mult_input_data(fileDir,token_to_idx,text_size):
    positive_text=read_file_data(fileDir+'positiveText/positiveText_cnn.lex',token_to_idx,text_size)
    original_text=read_file_data(fileDir+'originalText/originalText_cnn.lex',token_to_idx,text_size)
    negative_text=read_file_data(fileDir+'negativeText/negativeText_cnn.lex',token_to_idx,text_size)
    return original_text,positive_text,negative_text
def read_params(filePath):
    layer_one_W=nd.load(filePath+'conv_layer_one_W.txt')
    layer_one_B=nd.load(filePath+'conv_layer_one_B.txt')
    layer_two_W=nd.load(filePath+'conv_layer_two_W.txt')
    layer_two_B=nd.load(filePath+'conv_layer_two_B.txt')
    layer_three_W=nd.load(filePath+'conv_layer_three_W.txt')
    layer_three_B=nd.load(filePath+'conv_layer_three_B.txt')
    layer_four_W=nd.load(filePath+'conv_layer_four_W.txt')
    layer_four_B=nd.load(filePath+'conv_layer_four_B.txt')
    layer_connect_W=nd.load(filePath+'ful_layer_connect_W.txt')
    layer_connect_B=nd.load(filePath+'ful_layer_connect_B.txt')
    params=[layer_one_W,layer_one_B,layer_two_W,layer_two_B,layer_three_W,layer_three_B,layer_four_W,layer_four_B,layer_connect_W,layer_connect_B]
    return params

In [3]:
idx_to_token,token_to_idx,embedding_weight=read_embedding_params("data/params/word2Vec/")
embedding_dim=20
text_size=800

In [4]:
#params=init_params()
params=read_params('data/params/conv/')
#params_two=params[0]
#print(params[0][0])
conv_one_W=params[0][0]
conv_one_B=params[1][0]
conv_two_W=params[2][0]
conv_two_B=params[3][0]
conv_three_W=params[4][0]
conv_three_B=params[5][0]
conv_four_W=params[6][0]
conv_four_B=params[7][0]
full_connect_W=params[8][0]
full_connect_B=params[9][0]
params=[conv_one_W,conv_one_B,conv_two_W,conv_two_B,conv_three_W,conv_three_B,conv_four_W,conv_four_B,full_connect_W,full_connect_B]
#print(params)

In [5]:
embedding_weight=nd.array(embedding_weight[0])

In [ ]:
save_params(params,'data/params/conv/')

In [6]:
def conv_activation(w,b,z,ctx):
    w=w.as_in_context(ctx)
    b=b.as_in_context(ctx)
    sum=0;
    for t in z:
        sum+=t.sum()
    if sum==0:
        return 0
    #sum_two=(nd.dot(z,w)+b,ctx=ctx)
    #sum_two=sum_two.as_in_context(ctx)
    sum_two=nd.dot(z,w)+b
    for i in range(sum_two.shape[0]):
        for j in range(sum_two.shape[1]):
            sum_two[i][j]=max(sum_two[i][j],0)
   
    return sum_two
def conv_layer(X,windows_size,param_w,param_b,ctx):
    #windows_size=3
    #print(X.shape[0])
    Y=nd.zeros((X.shape[0],windows_size,param_w.shape[1]))
    Y=Y.as_in_context(ctx)
    X=X.as_in_context(ctx)
    param_w=param_w.as_in_context(ctx)
    params_b=param_b.as_in_context(ctx)
    for i in range(X.shape[0]):
        if(i+windows_size>=X.shape[0]):
            break
        Y[i]=conv_activation(param_w,param_b,X[i:i+windows_size],ctx)
    return Y
    
def myMax(X_one,X_two):
    sum_one=X_one.sum()
    sum_two=X_two.sum()
    if sum_one>sum_two:
        return X_one
    else :
        return X_two
def max_pool_layer(X,ctx):
    temp= int(X.shape[0]/2)
    Y=nd.zeros((temp,X.shape[1]))
    Y=Y.as_in_context(ctx)
    j=0
    for i in range(X.shape[0]-1):
        if i%2==0:
            Y[j]=myMax(X[i],X[i+1])
            j+=1
    return Y
def full_connect_layer(X,W,B,ctx):
    Y=nd.zeros(W.shape[1])
    Y=Y.as_in_context(ctx)
    X=X.as_in_context(ctx)
    W=W.as_in_context(ctx)
    B=B.as_in_context(ctx)
    Y=nd.dot(X.T,W).T+B
    return Y.tanh()

def dropout_layer(X,drop_prob):
    assert 0 <= drop_prob <=1
    keep_prob=1-drop_prob
    if keep_prob==0:
        return X.zeros_like()
    mask=nd.random.uniform(0,1,X.shape)<keep_prob
    return mask*X/keep_prob
def make_parameterDict(params):
    conv_one_W,conv_one_B,conv_two_W,conv_two_B,conv_three_W,conv_three_B,conv_four_W,conv_four_B,full_connect_W,full_connect_B=params
    paraDict=gluon.ParameterDict()
    
    para_one_W=gluon.Parameter('conv_one_W',shape=conv_one_W.shape,init=init.Xavier())
    para_one_B=gluon.Parameter('conv_one_B',shape=conv_one_B.shape,init=init.Xavier())
    para_one_W.initialize()
    para_one_B.initialize()
    paraDict._params['conv_one_W']=para_one_W
    paraDict._params['conv_one_B']=para_one_B
        
    para_two_W=gluon.Parameter('conv_two_W',shape=conv_two_W.shape,init=init.Xavier())
    para_two_B=gluon.Parameter('conv_two_B',shape=conv_two_B.shape,init=init.Xavier())
    para_two_W.initialize()
    para_two_B.initialize()
    paraDict._params['conv_two_W']=para_two_W
    paraDict._params['conv_two_B']=para_two_B
        
    para_three_W=gluon.Parameter('conv_three_W',shape=conv_three_W.shape,init=init.Xavier())
    para_three_B=gluon.Parameter('conv_three_B',shape=conv_three_B.shape,init=init.Xavier())
    para_three_W.initialize()
    para_three_B.initialize()
    paraDict._params['conv_three_W']=para_three_W
    paraDict._params['conv_three_B']=para_three_B
        
    para_four_W=gluon.Parameter('conv_four_W',shape=conv_four_W.shape,init=init.Xavier())
    para_four_B=gluon.Parameter('conv_four_B',shape=conv_four_B.shape)
    para_four_W.initialize()
    para_four_B.initialize()
    paraDict._params['conv_four_W']=para_four_W
    paraDict._params['conv_four_B']=para_four_B
        
    para_full_connect_W=gluon.Parameter('full_connect_W',shape=full_connect_W.shape,init=init.Xavier())
    para_full_connect_B=gluon.Parameter('full_connect_B',shape=full_connect_B.shape,init=init.Xavier())
    para_full_connect_W.initialize()
    para_full_connect_B.initialize()
    paraDict._params['full_connect_W']=para_full_connect_W
    paraDict._params['full_connect_B']=para_full_connect_B
    return paraDict

In [12]:
class CNNCodeRecheckTrainModel(nn.Block):
    def __init__(self,vocab_size,embed_size,embedding_weight,**kwargs):
        super(CNNCodeRecheckTrainModel,self).__init__(**kwargs)
        #参与训练嵌入层
        self.embedding=nn.Embedding(input_dim=vocab_size,output_dim=embed_size)
        self.embedding.weight.initialize(init=init.Xavier(),force_reinit=True)
        self.embedding.weight.set_data(embedding_weight)
        #不参与训练嵌入层
        #self.constant_embedding=nn.Embedding(input_dim=vocab_size,output_dim=embed_size)
        #申请梯度
        params=read_params('data/params/conv/')
        conv_one_W=params[0][0]
        conv_one_B=params[1][0]
        conv_two_W=params[2][0]
        conv_two_B=params[3][0]
        conv_three_W=params[4][0]
        conv_three_B=params[5][0]
        conv_four_W=params[6][0]
        conv_four_B=params[7][0]
        full_connect_W=params[8][0]
        full_connect_B=params[9][0]
        params=[conv_one_W,conv_one_B,conv_two_W,conv_two_B,conv_three_W,conv_three_B,conv_four_W,conv_four_B,full_connect_W,full_connect_B]
        for param in params:
            #print(param)
            param.attach_grad()
        ctx=d2l.try_gpu()
        #self.embedding=self.embedding.as_in_context(ctx)
        self.conv_one_W=conv_one_W
        self.conv_one_W=self.conv_one_W.as_in_context(ctx)
        #print(self.conv_one_W)
        self.conv_one_B=conv_one_B
        self.conv_one_B=self.conv_one_B.as_in_context(ctx)
        #print(self.conv_one_B)
        self.conv_two_W=conv_two_W
        self.conv_two_W=self.conv_two_W.as_in_context(ctx)
        
        self.conv_two_B=conv_two_B
        self.conv_two_B=self.conv_two_B.as_in_context(ctx)
        
        self.conv_three_W=conv_three_W
        self.conv_three_W=self.conv_three_W.as_in_context(ctx)
        
        self.conv_three_B=conv_three_B
        self.conv_three_B=self.conv_three_B.as_in_context(ctx)
        
        self.conv_four_W=conv_four_W
        self.conv_four_W=self.conv_four_W.as_in_context(ctx)
        
        self.conv_four_B=conv_four_B
        self.conv_four_B=self.conv_four_B.as_in_context(ctx)
        
        self.full_connect_W=full_connect_W
        self.full_connect_W=self.full_connect_W.as_in_context(ctx)
        
        self.full_connect_B=full_connect_B
        self.full_connect_B=self.full_connect_B.as_in_context(ctx)
        self.conv_layer=conv_layer
        self.max_pool_layer=max_pool_layer
        self.full_connect_layer=full_connect_layer
        self.dropout_layer=dropout_layer
        self.paraDict=make_parameterDict(params)
    def coll_para():
        print(paraDict)
        #return paraDict    
    def set_embedding_weight(self,embedding_weight):
        self.embedding.weight.set_data(embedding_weight)
    def forward(self,inputs,mode='train'):
        ctx=d2l.try_gpu()
        inputs_embeddings=nd.array([],ctx=ctx)
        if mode=='train':
            inputs_embeddings=nd.stack(nd.flatten(self.embedding(inputs[0])),nd.flatten(self.embedding(inputs[1])),nd.flatten(self.embedding(inputs[2])))
        else :
            inputs_embeddings=nd.stack(nd.flatten(self.embedding(inputs[0])),nd.flatten(self.embedding(inputs[1])))
        #print(inputs_embeddings)
        #input_embedding=inputs_embeddings[0]
        #sum=0;
        results=[]
        for input_embedding in inputs_embeddings:
            input_embedding=input_embedding.as_in_context(ctx)
            result=conv_layer(input_embedding,3,self.conv_one_W,self.conv_one_B,ctx)
            result=result.as_in_context(ctx)
            result=nd.flatten(result)
            print(result)
            result=max_pool_layer(result,ctx)
            result=conv_layer(result,3,self.conv_two_W,self.conv_two_B,ctx)
            result=nd.flatten(result)
            result=max_pool_layer(result,ctx)
            print(result)
            result=conv_layer(result,3,self.conv_three_W,self.conv_three_B,ctx)
            result=nd.flatten(result)
            result=max_pool_layer(result,ctx)
            print(result)
            result=conv_layer(result,1,self.conv_four_W,self.conv_four_B,ctx)
            result=nd.flatten(result)
            result=result.as_in_context(ctx)
            result=full_connect_layer(result,self.full_connect_W,self.full_connect_B,ctx)
            print(result)
            results.append(result)
        return results

In [8]:
def get_mult_file_data(sourcePath,negativePath,positivePath,token_to_idx,text_size):
    text=[]
    sourceText=read_file_data(sourcePath,token_to_idx,text_size)
    negativeText=read_file_data(negativePath,token_to_idx,text_size)
    positiveText=read_file_data(positivePath,token_to_idx,text_size)
    sourceText=nd.array(sourceText)
    positiveText=nd.array(positiveText)
    negativeText=nd.array(negativeText)
    text=nd.stack(sourceText,positiveText,negativeText)
    return text
def get_fileNames(dirPath):
    filenameSet=[]
    for dirpath,dirnames,filenames in os.walk(dirPath):
        for fileaname in filenames:
            filePath=os.path.join(dirpath,fileaname)
            if os.path.isfile(filePath):
                if filePath.endswith(".txt"):
                       filenameSet.append(fileaname)
    #print(filenameSet)
    sorted(filenameSet)
    return filenameSet
def get_all_file_in_dir(dirPath):
    filePaths=[]
    for dirpath,dirnames,filenames in os.walk(dirPath):
        for filename in filenames:
           # filenameSet.append(filenames)
            #print(os.path.join(dirpath,filename))
            filePath=os.path.join(dirpath,filename)
            if os.path.isfile(filePath):
                if filePath.endswith(".txt"):
                    #print(filePath)
                    filePaths.append(filePath)
    sorted(filePaths)
    return filePaths
def load_train_data(net,token_to_idx,text_size):
    #filenames=get_fileNames("C:/Users/lwq/Desktop/trainData/source")
    #print(filenames)
    sourceFiles=get_all_file_in_dir("C:/Users/lwq/Desktop/trainData/source")
    positiveFiles=get_all_file_in_dir("C:/Users/lwq/Desktop/trainData/positive")
    negativeFiles=get_all_file_in_dir("C:/Users/lwq/Desktop/trainData/negative")
    texts=[]
    for idx in range(len(sourceFiles)):
        #print(sourceFiles[idx])
        #print(negativeFiles[idx])
        #print(positiveFiles[idx])
        text=get_mult_file_data(sourceFiles[idx],negativeFiles[idx],positiveFiles[idx],token_to_idx,text_size)
        #print(text)
        texts.append(text)
    return texts

In [15]:
original_text,positive_text,negative_text=read_mult_input_data("data/input_data/",token_to_idx,text_size)
original_text=nd.array(original_text)
positive_text=nd.array(positive_text)
negative_text=nd.array(negative_text)
text=nd.stack(original_text,positive_text,negative_text)

In [16]:
lr,num_epochs=0.001,5
net=CNNCodeRecheckTrainModel(len(idx_to_token),embedding_dim,embedding_weight)
trainer=gluon.Trainer(net.paraDict,'adam',{'learning_rate':lr})

In [ ]:
time_start=time.time()
y=net(text)
time_end=time.time()
print('花费时间%.3fs'%(time_end-time_start))

In [ ]:
net=CNNCodeRecheckTrainModel(len(idx_to_token),embedding_dim,embedding_weight)
texts=load_train_data(net,token_to_idx,text_size)
text_embeddings=[]
sum=0
time_total=0
for text in texts:
    time_start=time.time()
    embedd=net(text)
    text_embeddings.append(embedd)
    time_end=time.time()
    cost_time=time_end-time_start
    time_total+=cost_time
    print('样本%d计算花费时间%.3fs'%(sum,cost_time))
    sum+=1
print('%d份样本花费时间%.3fs'%(sum,time_total))